In [9]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"
dirNameInput = os.getcwd() + '/Word2VecData_Input'
dirNameOutput = os.getcwd() + '/Word2VecData_Output'

from sklearn.model_selection import train_test_split

Word2Vec_model = None

X = None
Y = None
X_Actual = None
Y_Actual = None
X_Simulated = None
Y_Simulated = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None
result = []


In [10]:
def createDataFiles():
    import time
    start = time.time()
    stride = 2
    
    fileNamesActual = ['autopark.dat', 'drive.dat', 'idle.dat']
    
    # To only work with simulated data start commenting from here
    fileInput = open("Word2VecData_Input\Word2VecDataActual.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataActual.txt", "w")
    

    for i in range(len(fileNamesActual)):
        with open(fileNamesActual[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        value = int(0.8 * len(content))
        
        value = len(content)

        cntCntr = 0
        while cntCntr < value:
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < value:
                parts = content[cntCntr].split(",")

                # this is the whole message
                DATA = parts[3].split(":")[1].replace(" ", "")

                currMessagePackets += str(DATA) + " "

                cntCntr += 1
                iteration += 1

            if cntCntr < value:
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(i) + "\n")
    fileInput.close()
    fileOutput.close()
    # To only work with Simulated data end commenting till here

    # To only work with actual data start commenting from here
    # For Simulated data
    fileInput = open("Word2VecData_Input\Word2VecDataSimulated.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['New_DRIVE_Data.txt', 'New_IDLE_Data.txt', 'DRIVE.rtf', 'IDLE.rtf']
    arrSimulated = [1, 2, 1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < len(content):
                # Remove the noise from the files
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    # To only work with actual data end commenting till here
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)

In [11]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=200, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [12]:
def train():
    createDataFiles()
    createWord2VecModels()



In [13]:
train()

Awesome !!! File processing done !!!
Total Time for file processing --->  1.2860157489776611


Training Successful for Word2Vec Model!!!
Total Time for Word2Vec model ->  2.465160608291626


In [16]:
# Convert the data into Numpy Arrays
start = time.time()
Word2Vec_model = Word2Vec.load(modelfileName)
x_train = None
def getNumpyArray():
    global X
    global Y
    global X_Actual
    global Y_Actual
    global X_Simulated
    global Y_Simulated
    global dirNameInput
    X = None
    Y = None
    result = []
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            for i in range(0, len(parts)):
                result.append(np.array(Word2Vec_model.wv[parts[i]]))
        if "Actual" in fName: 
            X_Actual = np.array(result)
            result = []
        else:
            X_Simulated = np.array(result)
            result = []
    
    X = np.concatenate((X_Actual, X_Simulated), axis=0)
    #X = np.array(result)
                

    result = []
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            for i in range(0,5):
                result.append(currV)
        if "Actual" in fName:
            Y_Actual = np.array(result)
            result = []
        else:
            Y_Simulated = np.array(result)
            result = []
                
    Y = np.concatenate((Y_Actual, Y_Simulated), axis=0)
    #Y = np.array(result)
    print(X_Actual.shape)
    print(Y_Actual.shape)
    print(X_Simulated.shape)
    print(Y_Simulated.shape)
    print(X.shape)    
    print(Y.shape)    
    end = time.time()
    print("total time -> ", end -start)


#    LSTM

In [17]:
getNumpyArray()

(27940, 200)
(27940,)
(105945, 200)
(105945,)
(133885, 200)
(133885,)
total time ->  3.0042128562927246


In [18]:
#execute LSTM model

def LSTMModel(epoch, X_array, Y_array):
    start = time.time()   
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(5, return_sequences=True,input_shape=(200,1)))
    model.add(tf.keras.layers.LSTM(5 ,input_shape=(200,1)))
    model.add(tf.keras.layers.Dense(1, activation='relu'))

    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    
    x_train, x_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=0.30, random_state=42)
    
    model.fit(x_train, y_train, batch_size = 30 , epochs=epoch, verbose = 1)

    val_loss, val_acc = model.evaluate(x_test, y_test)
    print("Test score: ", val_loss)
    print("Test accuracy: ", val_acc)

    end = time.time()
    print("Total Time -> ", (end - start))




# For actual data

In [19]:
#split the data and train the model
X_Actual = X_Actual[:10000]
Y_Actual = Y_Actual[:10000]
X_Actual = X_Actual.reshape(10000,200,1)
print(X_Actual.shape)
print(Y_Actual.shape)

from sklearn.model_selection import train_test_split
LSTMModel(10, X_Actual, Y_Actual)

(10000, 200, 1)
(10000,)
Epoch 1/10
7000/7000 [==============================] - 125s 18ms/step - loss: 0.2596 - acc: 0.9270
Epoch 2/10
 540/7000 [=>............................] - ETA: 2:09 - loss: 0.2715 - acc: 0.9222

KeyboardInterrupt: 

# For simulated data

In [ ]:
#split the data and train the model
X_Simulated = X_Simulated[:10000]
Y_Simulated = Y_Simulated[:10000]
X_Simulated = X_Simulated.reshape(10000,200,1)
print(X_Simulated.shape)
print(Y_Simulated.shape)

from sklearn.model_selection import train_test_split
LSTMModel(10, X_Simulated, Y_Simulated)

# For actual + simulated data

In [47]:
#split the data and train the model
X = X[:10000]
Y = Y[:10000]
X = X.reshape(10000,200,1)
print(X.shape)
print(Y.shape)

from sklearn.model_selection import train_test_split
LSTMModel(10, X, Y)

(10000, 200, 1)
(10000,)
Epoch 1/10
7000/7000 [==============================] - 122s 17ms/step - loss: 0.2763 - acc: 0.9270
Epoch 2/10
7000/7000 [==============================] - 114s 16ms/step - loss: 0.2405 - acc: 0.9270
Epoch 3/10
7000/7000 [==============================] - 108s 15ms/step - loss: 0.2355 - acc: 0.9270
Epoch 4/10
 750/7000 [==>...........................] - ETA: 1:35 - loss: 0.1875 - acc: 0.9373

KeyboardInterrupt: 

In [ ]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()